In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling as pp
from sklearn.ensemble import ExtraTreesClassifier

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/covid19-global-forecasting-week-1/submission.csv
/kaggle/input/covid19-global-forecasting-week-1/test.csv
/kaggle/input/covid19-global-forecasting-week-1/train.csv


In [2]:
train = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-1/train.csv")
test = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-1/test.csv")

# Using Pandas Profiling for analysis

In [3]:
pp.ProfileReport(train)

In [4]:
train = train.drop(["Province/State"], axis = "columns")

### It is found that aruba is the country that has missing Lat and Long so I took the real value of it.

In [5]:
train["Lat"] = train["Lat"].fillna(12.521)
train["Long"] = train["Long"].fillna(69.968)
train["Date"] = train["Date"].apply(lambda x: x.replace("-",""))
train["Date"] = train["Date"].astype(int)

In [6]:
test = test.drop(["Province/State"], axis = "columns")
test["Date"] = test["Date"].apply(lambda x: x.replace("-", ""))
test["Date"] = test["Date"].astype(int)
test["Lat"] = test["Lat"].fillna(12.521)
test["Long"] = test["Long"].fillna(69.968)

In [7]:
x = train[["Lat", "Long", "Date"]]
y = train[["ConfirmedCases", "Fatalities"]]

In [8]:
from sklearn.ensemble import ExtraTreesClassifier
model1 = ExtraTreesClassifier()

model1.fit(x ,y["ConfirmedCases"])


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [9]:
from sklearn.ensemble import ExtraTreesClassifier
model4 = ExtraTreesClassifier()

model4.fit(x ,y["Fatalities"])



ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [10]:
test_x = test[["Lat", "Long", "Date"]]
cases_predict = model1.predict(test_x)
fat_predict = model4.predict(test_x)
ConfirmedCases = pd.DataFrame(cases_predict)
Fatalities = pd.DataFrame(fat_predict)
test = pd.concat([test, ConfirmedCases, Fatalities], axis = "columns")

In [11]:
test.head()

,ForecastId,Country/Region,Lat,Long,Date,0,0
0,1,Afghanistan,33.0,65.0,20200312,7.0,0.0
1,2,Afghanistan,33.0,65.0,20200313,7.0,0.0
2,3,Afghanistan,33.0,65.0,20200314,11.0,0.0
3,4,Afghanistan,33.0,65.0,20200315,16.0,0.0
4,5,Afghanistan,33.0,65.0,20200316,16.0,0.0


In [12]:
test = test.drop(["Country/Region", "Lat", "Long", "Date"], axis = "columns")
test.columns = ["ForecastId", "ConfirmedCases", "Fatalities"]

In [13]:
test.to_csv("submission.csv", index = False)